In [2]:
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Point, LineString

In [3]:
# data_frames = []

# for i in range(1, 16):
#     file_name = f"../res/result{i}.csv"
#     df = pd.read_csv(file_name)
#     data_frames.append(df)

# # 使用 concat 函数将所有 DataFrame 合并为一个
# result = pd.concat(data_frames)

# result.to_csv("../res/sample_taxi_region.csv")
df = pd.read_csv("../res/sample_taxi_region.csv")

In [4]:
# 按照出租车编号对数据进行分组
df_grouped = df.groupby('taxi_id')

In [16]:
# from datetime import datetime
# # 定义一个将时间字符串转换为datetime类型的函数
# def convert_time(time_string):
#     return datetime.strptime(time_string, '%H:%M:%S')

# # 使用apply行向量函数将整个时间列转换为datetime类型
# df_grouped['time'] = df_grouped['time'].apply(lambda x: convert_time(x))
# print(df_grouped.head(1000))
print(type(df_grouped))
df_grouped_slice = df_grouped.head(1000)
print(type(df_grouped_slice))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
<class 'pandas.core.frame.DataFrame'>


In [18]:
# 分别处理每个出租车的订单
orders = []
df_sorted = df_grouped.apply(lambda x: x.sort_values('time'))
order_started = False
order_start_lat = None
order_start_lon = None
order_end_lat = None
order_end_lon = None
order_start_district = None
order_start_block = None
order_end_district = None
order_end_block = None
for _, row in df_sorted.iterrows():
    if row['is_passenger'] == 1 and not order_started:
        # 如果订单开始，记录起始点位置和时间
        order_start_lat = row['lat']
        order_start_lon = row['lon']
        order_start_time = row['time']
        order_start_district = row['region']
        order_start_block = row['block_id']
        order_started = True
    elif row['is_passenger'] == 0 and order_started:
        # 如果订单结束，记录终点位置，添加订单记录
        order_end_lat = row['lat']
        order_end_lon = row['lon']
        order_end_time = row['time']
        order_end_district = row['region']
        order_end_block = row['block_id']
        orders.append((row['taxi_id'], order_start_lat, order_start_lon, order_end_lat, order_end_lon, order_start_time, order_end_time, order_start_district, order_start_block, order_end_district, order_end_block))
        order_started = False

In [20]:
# 将订单记录整合到DataFrame中
orders_df = pd.DataFrame(orders, columns=['taxi_id', 'start_lat', 'start_lon', 'end_lat', 'end_lon', 'start_time', 'end_time', 'start_district', 'start_block', 'end_district', 'end_block'])
orders_df.to_csv('../res/orders.csv', index=False)